In [25]:
import pandas as pd
import os
from os import path
import glob
import numpy as np
from nltk.corpus import stopwords
import nltk
from collections import Counter
import dask.dataframe as dd
import math
from pattern.nl import tag

In [2]:
stopz = [word.split() for word in open("C:\\Users\\Ruben\\Documents\\Artikelen\\Joris\\stopwords-nl.txt", 'r', encoding = "utf-8").readlines()]

In [3]:
## Import Master DF
os.chdir("C://Users//Ruben//Documents//Scriptie//Data//bigrams//output")
df = dd.read_csv('total-bigrams.csv', dtype='str')

In [4]:
## Load Buitenlandsche Bigrams
bgdf = pd.read_csv('C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data//bigrams_buitenlandsche.csv', encoding = 'utf-8')

In [9]:
test = bgdf.iloc[:,[0,3]]
test.columns = test.columns.astype(str)
test[test['1817'] >0]

,ngram,1817
166,ambassadeurs,2.0
293,berigten,303.0
327,betrekkingen,9.0
359,bezittingen,7.0
499,dagbladen,6.0
567,effekten,4.0
781,gezantschappen,2.0
825,handel,2.0
1273,ministers,3.0
1288,mogendheden,6.0


In [10]:
def GetListWords():
    dwy = dict()
    
    for i in list(range(1,100)):
        tdf = bgdf.iloc[:,[0,i]]
        tdf.columns = tdf.columns.astype(str)
        y = str(tdf.columns[1])
        tdf = tdf[tdf[y] >0]
        tdf = tdf.reset_index(drop = True)
        tdf = list(tdf['ngram'])
        
        dwy.update({y:list(tdf)})
    return dwy

In [16]:
reference_list = list(GetListWords().values())
total_list = list(GetListWords().values())
total_list = list(set([item for sublist in total_list for item in sublist]))
total_list = [l for l in total_list if len(l) > 2]

In [17]:
def GetBigrams(input_list):
    subset = df[df['ngram'].str.split().str[1].isin(input_list)].compute()
    subset = subset.reset_index(drop=True)
    subset['count'] = subset['count'].astype(float)
    subset = subset.groupby(['ngram','year']).agg('sum')
    subset = subset.stack().reset_index().drop('level_2', axis = 1)
    subset.columns = ['ngram', 'year', 'count']
    subset = subset.dropna()
    subset = subset.pivot(index='ngram', columns='year', values='count')
    subset = subset.fillna(0)
    subset.columns = subset.columns.astype(str)
    
    return subset

In [18]:
dfr = GetBigrams(total_list)

In [23]:
dfr['ngram'] = dfr.index

In [41]:
def GetListAdjectives(input_df, freq_thres):
    dwy = dict()
    
    for i in list(range(1815,1914)):
        yl = list()
        tdf = input_df[input_df[str(i)] > freq_thres]
        for key,value in Counter(list(tdf.ngram.str.split().str[0])).items():
            if value > 10 and tag(key)[0][1] == 'JJ' and len(key) > 2 and key not in stopz:
                tmp = [key, value]
                yl.append(tmp)
            
        
        dwy.update({i:yl})
    return dwy

In [42]:
lad = GetListAdjectives(dfr, 5)

In [49]:
lad_df = pd.DataFrame()

for key, value in lad.items():
    for i in value:
        tmp = pd.DataFrame([key], columns = ['year'])
        tmp['word'] = i[0]
        tmp['count'] = i[1]
        lad_df = lad_df.append(tmp)

In [51]:
lad_df

,year,word,count
0,1815,armee,25
0,1815,bijzonder,11
0,1815,dadelijk,18
0,1815,dezelve,58
0,1815,eenige,29
0,1815,fransche,16
0,1815,geene,15
0,1815,geheel,16
0,1815,groote,14
0,1815,hetzelve,34


In [59]:
lad_df.columns = ['word', 'year', 'count']
lad_df = lad_df.pivot(index='word', columns='year', values='count')
lad_df = lad_df.fillna(0)
lad_df.columns = lad_df.columns.astype(str)

In [61]:
os.chdir('C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data')
lad_df.to_csv('adjective_distribution_buitenlandsche.csv')


In [37]:
lad_df.columns = ['word', 'year']

In [40]:
lad_df[lad_df.word == 'groote']

,word,year
8,groote,1815
7,groote,1816
4,groote,1818
5,groote,1822
6,groote,1824
8,groote,1825
9,groote,1826
11,groote,1827
8,groote,1828
12,groote,1829
